# Chapter 3 - The Agent-Environment Interface

## Gridworld - Random Policy

Here we replicate figure 3.5 (pg. 67) of Sutton-Barto, shown below.


The system dynamics are fairly straightforward. You are in a square as your current state and you can move up, down, left, or right over a uniform probability distribution (e.g. the probability of a move in any individual direction is 0.25). If you move off the grid you receive a reward of -1 and your next state is your current state. If you find yourself in states A or B you will move deterministically to A' or B', respectively, and receive a reward of 10 or 5, respectively. Any other move in any other direction has a reward of 0. We discount the moves by a factor of 0.9. 

Graphically we show the reward dynamics and random state value functions below:

![alt text](pic1.png)

In [4]:
import numpy as np
    
#Define system dynamics
def movestate(inputstate, nextstate):
    if inputstate == [0,1]:
        reward = 10
        prob = .25
        outstate = [4,1]
    elif inputstate == [0,3]:
        reward = 5
        prob = .25
        outstate = [2,3]
    elif nextstate[0] > 4 or nextstate[0] < 0 or nextstate[1] > 4 or nextstate[1] < 0:
        outstate = inputstate
        prob = .25
        reward = -1
    else:
        outstate = nextstate
        prob = .25
        reward = 0
    return outstate, prob, reward

In [7]:
grid = np.zeros((5,5))   

#loop until convergence
while True:
    
    newgrid = np.zeros((5,5))  

    for i in range(5):
        for j in range(5):
            #Up
            Outstate, Prob, Reward = movestate([i,j], [i+1, j])
            newgrid[i, j] += Prob * (Reward + .9 * grid[Outstate[0], Outstate[1]])
            # Plus Down
            Outstate, Prob, Reward = movestate([i,j], [i-1, j])
            newgrid[i, j] += Prob * (Reward + .9 * grid[Outstate[0], Outstate[1]])
            # Plus Right
            Outstate, Prob, Reward = movestate([i,j], [i, j+1])
            newgrid[i, j] += Prob * (Reward + .9 * grid[Outstate[0], Outstate[1]])
            # Plus Left
            Outstate, Prob, Reward = movestate([i,j], [i, j-1])
            newgrid[i, j] += Prob * (Reward + .9 * grid[Outstate[0], Outstate[1]])
    #continue to loop until we converge
    if np.sum(np.abs(grid - newgrid)) < 1e-4:
        print(np.round(newgrid,1))
        break
    grid = newgrid

[[ 3.3  8.8  4.4  5.3  1.5]
 [ 1.5  3.   2.3  1.9  0.5]
 [ 0.1  0.7  0.7  0.4 -0.4]
 [-1.  -0.4 -0.4 -0.6 -1.2]
 [-1.9 -1.3 -1.2 -1.4 -2. ]]


## Gridworld - Optimal Policy

Next we evaluate the optimal policy and recreate figure 3.8 (pg. 73).

![alt text](pic2.png)

    
Here the system dynamics are the same, but instead of taking the expected reward of each state we deterministically move to the state with the highest expected reward. The figure on the right is the optimal movement policy, while the center figure is the optimal policy's expected reward. 

In [8]:
grid = np.zeros((5,5))   

while True:
    
    newgrid = np.zeros((5,5))  

    for i in range(5):
        for j in range(5):
            value = []
            
            Outstate, Prob, Reward = movestate([i,j], [i+1, j])
            value.append(Reward + .9 * grid[Outstate[0], Outstate[1]])
            
            Outstate, Prob, Reward = movestate([i,j], [i-1, j])
            value.append(Reward + .9 * grid[Outstate[0], Outstate[1]])
            
            Outstate, Prob, Reward = movestate([i,j], [i, j+1])
            value.append(Reward + .9 * grid[Outstate[0], Outstate[1]])
            
            Outstate, Prob, Reward = movestate([i,j], [i, j-1])
            value.append(Reward + .9 * grid[Outstate[0], Outstate[1]])
            
            newgrid[i,j] = np.max(value)
            
    if np.sum(np.abs(grid - newgrid)) < 1e-4:
        print(np.round(newgrid, 1))
        break
    grid = newgrid

[[ 22.   24.4  22.   19.4  17.5]
 [ 19.8  22.   19.8  17.8  16. ]
 [ 17.8  19.8  17.8  16.   14.4]
 [ 16.   17.8  16.   14.4  13. ]
 [ 14.4  16.   14.4  13.   11.7]]


### Chapter Highlights

1. At time $t$, the agent receives a state $S \in \mathcal{S}$, performs an action $A_{t} \in \mathcal{A}(S_{t})$, and receives some reward $R_{t+1} \in \mathcal{R} \subset \mathbb{R}$, then transitions to a new state$ S_{t+1}$.

2. At each step, the agent selects a "next action" $a$ based on a policy $\pi_{t}$, defined as $\pi_{t}(a \mid s)$, which is the probability that a specific action is taken based on the current state $s$.

3. With each timestep, the agent intends to maximize a goal, or reward. This reward communicates *what* you want achieved, not necessarily *how* you want it achieved. 

4. We define the goal as a miximization of the expected return $G_{t}$, which is defined as a function of the sum of the future rewards discounted by some *discount rate* $\gamma$, where $0 \leq \gamma \leq 1$.

5. Formally, we define the *discounted return* to be $G_{t} = R_{t+1} + R_{t+2} + ... = \sum\limits_{k=0}^\infty \gamma^{k}R_{t+k+1} $

 - It follows that if $\gamma < 1$ and the rewards are bounded, this sum will converge to a finite value on an infinite horizon.

6. A process is considered *markovian* if the future states and actions of the process at time $t$ are dependent only on the state and actions of time $t-1$. Or, more formally, $p(s',r \mid s,a) =  $ Pr$\left\{ S_{t+1} = s', R_{t+1} = r \mid S_{t} = s, A_{t} = a \right\}$. This is important because it implies that the present state\reward\action tuples are in no way dependent on previous values of the same variables. 

7. A value function, $v_{\pi}(s)$ is used to estimate *how good* it is for the agent to be in the given state, and is defined formally as $v_{\pi}(s) = \mathbb{E}_{\pi} \big[ G_{t} \mid S_{t} = s \big]$

8. The action value function is used to estimate the vlaue of taking a particular action $a$ in state $s$ given policy $\pi$ as $v_{\pi}(s,a) = \mathbb{E}_{\pi} \big[ G_{t} \mid S_{t} = s, A_{t} = a \big]$

9. The value of a current state in relation to the value of its possible future successor states can be defined through the bellman equation, which is defined for the value function as $v_{\pi}(s) = \sum\limits_{a} \pi(a \mid s) \sum\limits_{s',r} p(s',r \mid s,a) \big[ r + \gamma v_{\pi}(s')\big]$. 
 
 - This can be described in words as finding the current reward plus the discounted future reward (one step ahead), weighing it by the probability of that particular outcome occurring, the summing over all the possibilities of all the different outcomes.
 
 - This same equation also maximizes the action-value function described above.
 
10. in a reinforcement learning task, we seek to maximise the return over a period of time, or the value over a period of time, on a particular policy $\pi(A \mid s)$

 - The *optimal value function* is defined as $v_{*}(s) = \underset{\pi}{\operatorname{max}} v_{\pi}(s,a)$
 
11. Optimal policies share the same *optimal action-value function*, which is defined as $q_{*}(s,a) = \underset{\pi}{\operatorname{max}} q_{\pi}(s,a)$ and gives the expected return for taking action $a$ instate $s$ on policy $\pi$

12. The key difference to note between the average bellman equations above and the optimal bellman equations are the max operator, which instead seeks to maximize the expected return instead of simply taking an average over all the different possible transition states. This results in a deterministic transition as opposed to a probabilistic one, therefore we remove the expectation notation.
